# Texture

In [ ]:
# please provide your OPENAI_KEY
OPENAI_KEY = "sk-<your-openai-key>"

## Related Functions

In [ ]:
from experiment import Logger
import json
import re
import random
import requests
import base64

def run_local_vision_request(text, image_urls, temperature=0):
    # Function to encode the image
    def encode_image(image_path):
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
    api_key = OPENAI_KEY

    messages = [
        {
            "role": "user",
            "content": [{"type": "text", "text": text},
            ]
        }
    ]

    for image_path in image_urls:
        # Getting the base64 string
        base64_image = encode_image(image_path)
        messages[0]["content"].append({
            "image_url": f"data:image/jpeg;base64,{base64_image}"
        })

    headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
    }

    payload = {
        "model": "gpt-4-vision-preview",
        "messages": messages,
        "max_tokens": 2048,
        "temperature": temperature,
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    return response.json()

def run_test(name, description, prompt, image_urls, logger, times=1):
    reason_list = []
    score_list = []
    while len(score_list) < times:
        shuffled_list = image_urls.copy()
        random.shuffle(shuffled_list)
        index = [image_urls.index(x) for x in shuffled_list]
        try:
            response = run_local_vision_request(
                text=prompt, 
                image_urls=shuffled_list,
            )
            matched_content = re.search(r'```json([\s\S]*?)```', response["choices"][0]["message"]["content"])
            response_list = json.loads(matched_content.group(1))
            score = sorted(response_list, key=lambda x: index[response_list.index(x)])
            reason_list.append(response["choices"][0]["message"]["content"])
            score_list.append(sorted(response_list, key=lambda x: index[response_list.index(x)]))
            logger.log(name, description, prompt, shuffled_list, response["choices"][0]["message"]["content"], score)
        except Exception as e:
            print(e)
    return reason_list

class Logger:
    def __init__(self, file_path):
        self.file_path = file_path
        
    def log(self, name, description, prompt, image_urls, response, score):
        json_data_to_add = {"name": name, "description": description, "prompt": prompt, "image_urls": image_urls, "response": response, "score": score}

        with open(self.file_path, 'r') as file:
            data = json.load(file)

        data.append(json_data_to_add)

        with open(self.file_path, 'w') as file:
            json.dump(data, file, indent=4)
logger = Logger("./texture.json")

## Experiment aesthetic

### Prompt Preparation

In [ ]:
ROLE = "You are an average user. "
TASK = "You are an average user. You need to rate the aesthetics of each visualization with iconic textures using a 7-point Likert scale via the 5 items of the BeauVis scale. BeauVis scale consists of five items, 'enjoyable', 'likable', 'pleasing', 'nice' and 'appealing'. And you also added 1 item to assess the degree to which they perceived a vibratory effect (Also using a 7-point Likert scale). Vibratory effect is an optical illusion making patterns seem unstable, linked to the Moiré effect. Then you also need to rank the four visualizations you have just evaluated based on your overall preference. You need to give the ranking of each image in order (1, 2, 3, ...)."
JSON_FORMAT = 'Please give an additional scoring result in json format at the end of your answe, like ```json[{{"enjoyable": score, "likable": score, "pleasing": score, "nice": score, "appealing": score, "vibratory_effect": score, "rank": rank }}, ...]```. '

### Data Preparation

In [ ]:
file_path = "./experiment_material/supp_materials/Experiment2/stimuli/"
type_list = ["BG", "BI", "MG", "MI", "PG", "PI"]
index_list = ["1", "2", "3", "4"]

### Task Execution

In [ ]:
name = "texture_experiment_aesthetic"
for _type in type_list:
    image_urls = [file_path + _type + index + ".png" for index in index_list]
    description = _type
    prompt = ROLE + \
        TASK + \
        JSON_FORMAT + \
        "DON'T say sorry or you cannot. YOU CAN."
    run_test(
        name=name,
        description=description,
        prompt=prompt,
        image_urls=image_urls,
        logger=logger,
        times=5)

## Experiment readable

### Prompt Preparation

In [ ]:
ROLE = "You are an average user. "
TASK = "Describe three visualizations with unicolor, geometric, and iconic textures. You need to rate the 5 items of the BeauVis scale and an additional readability item on a 7-point Likert scale. BeauVis scale consists of five items, 'enjoyable', 'likable', 'pleasing', 'nice' and 'appealing'. "
JSON_FORMAT = 'Please give an additional scoring result in json format at the end of your answe, like ```json[{{"enjoyable": score, "likable": score, "pleasing": score, "nice": score, "appealing": score, "readability": score}}, ...]```. '

### Data Preparation

In [ ]:
file_path = "./experiment_material/supp_materials/Experiment3/stimuli/"
vis_list = ["bar", "pie"]
type_list = ["color", "geo", "icon"]
index_list = ["1", "2", "3", "4", "5", "6", "7", "8", "9"]

### Task Execution

In [ ]:
name = "texture_experiment_readable"
for vis in vis_list:
    for index in index_list:
        image_urls = [file_path + vis + "_" + _type + index + ".png" for _type in type_list]
        description = "parallel_experiment_" + vis
        prompt = ROLE + \
            TASK + \
            JSON_FORMAT + \
            "DON'T say sorry or you cannot. YOU CAN."
        run_test(
            name=name,
            description=description,
            prompt=prompt,
            image_urls=image_urls,
            logger=logger,
            times=5)